In [1]:
from hugchat import hugchat
from hugchat.login import Login
from langchain.llms.base import LLM
from typing import Optional, List, Mapping, Any
from time import sleep

## Mnist Langchain

In [29]:
from langchain.llms.base import LLM
from typing import Optional, List, Mapping, Any

class CustomLLM(LLM):
  n:int
  
  @property
  def _llm_type(self) -> str:
    return "custom"

  def _call(self,prompt:str,stop:Optional[List[str]]=None) -> str:
    if stop is not None:
      raise ValueError("stop kwargs are not permitted")
    return prompt[:self.n]
  
  @property
  def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {"n": self.n}

In [31]:
llm = CustomLLM(n=111)
llm("222")

'222'

## Hugchat Langchain

In [48]:

class HuggingChat(LLM):
    
    history_data: Optional[List] = []
    chatbot : Optional[hugchat.ChatBot] = None
    conversation : Optional[str] = ""
    email : Optional[str]
    psw : Optional[str]
    #### WARNING : for each api call this library will create a new chat on chat.openai.com
    
    
    @property
    def _llm_type(self) -> str:
        return "custom"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        if stop is not None:
            pass
            #raise ValueError("stop kwargs are not permitted.")
        #token is a must check
        if self.chatbot is None:
            if self.email is None and self.psw is None:
                ValueError("Email and Password is required, pls check the documentation on github")
            else: 
                if self.conversation == "":
                    sign = Login(self.email, self.psw)
                    cookies = sign.login()

                    # Save cookies to usercookies/<email>.json
                    sign.saveCookiesToDir()

                    # Create a ChatBot
                    self.chatbot = hugchat.ChatBot(cookies=cookies.get_dict()) 
                else:
                    raise ValueError("Something went wrong")
            
        
        # sleep(5)
        data = self.chatbot.query(prompt, temperature=0.5, stream=False)
    
        
        #add to history
        # self.history_data.append({"prompt":prompt,"response":data})    
        # print(data)
        # print(data.get_final_text())
        # data.get_final_text()
        return str(data)

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {"model": "HuggingCHAT"}






In [49]:

llm = HuggingChat(email = "wangchenhao111@gmail.com" , psw = "Wch13892845832") #for start new chat


# print(llm("Hello, how are you?"))
# print(llm("what is AI?"))
# print(llm("Can you resume your previus answer?")) #now memory work well

In [43]:
llm("Hello, how are you?")
# llm.generate("Hello, how are you?")

" I don't have feelings like humans do, so I can't say that I'm feeling any particular way. However, I'm here to help answer your questions and assist with tasks to the best of my abilities. How may I be of service today?"

In [51]:
print(llm("what is AI?"))

 Artificial intelligence (AI) refers to the ability of machines or computer programs to mimic intelligent human behavior, such as learning, problem-solving, decision-making, perception, and language understanding. It involves the use of algorithms, data, and hardware to create systems that can perform tasks that typically require human intelligence, such as recognizing patterns, making predictions, and adapting to new situations.

There are many different types of AI, including:

1. Narrow or weak AI: This type of AI is designed to perform a specific task, such as facial recognition, language translation, or playing chess. Narrow AI is trained on a specific dataset and is not capable of general reasoning or decision-making.
2. General or strong AI: This type of AI is designed to perform any intellectual task that a human can. General AI has the ability to reason, learn, and improve its performance over time. It is still in development and has not yet been achieved.
3. Superintelligence

In [52]:
print(llm("Can you resume your previus answer?")) 

 Sure, my previous answer was:

"I apologize for the confusion. It seems that there is a problem with the prompt you provided. The prompt is not clear or specific enough to provide a useful response. Can you please rephrase or provide more context so I can better understand what you are asking and provide a helpful answer?"


## Langchain

In [27]:
import time
import logging
import requests
from typing import Optional, List, Dict, Mapping, Any
import langchain
from langchain.llms.base import LLM
from langchain.cache import InMemoryCache

In [28]:

logging.basicConfig(level=logging.INFO)
# 启动llm的缓存
langchain.llm_cache = InMemoryCache()


In [ ]:
class ChatGLM(LLM):
    history_data: Optional[List] = []
    chatbot : Optional[hugchat.ChatBot] = None
    conversation : Optional[str] = ""
    email : Optional[str]
    psw : Optional[str]
    # 模型服务url
    url = "http://127.0.0.1:8595/chat"

    @property
    def _llm_type(self) -> str:
        return "chatglm"

    def _construct_query(self, prompt: str) -> Dict:
        """构造请求体
        """
        query = {
            "human_input": prompt
        }
        return query

    @classmethod
    def _post(cls, url: str,
        query: Dict) -> Any:
        """POST请求
        """
        _headers = {"Content_Type": "application/json"}
        with requests.session() as sess:
            resp = sess.post(url, 
                json=query, 
                headers=_headers, 
                timeout=60)
        return resp

    
    def _call(self, prompt: str, 
        stop: Optional[List[str]] = None) -> str:
        """_call
        """
        # construct query
        query = self._construct_query(prompt=prompt)

        # post
        resp = self._post(url=self.url,
            query=query)
        
        if resp.status_code == 200:
            resp_json = resp.json()
            predictions = resp_json["response"]
            return predictions
        else:
            return "请求模型" 
    
    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters.
        """
        _param_dict = {
            "url": self.url
        }
        return _param_dict